In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# from AlgoTestFactory import *

# altf1 = AlgoTestFactory([0, 10000], 15, savePath="Combined_4_test2", storeAccuracies=True)

# altf1.run()

In [3]:
from SktWsegRWR_utf8 import *
import pickle
import ProbData
from ProbModels import *
import multiprocessing
import math
import json

Dataloader Started[Prob]...
Dataloader Finished[Prob]...


In [4]:
pb = ProbModels(fullCo_oc_mat = ProbData.fullCo_oc_mat, unigram_counts = ProbData.unigram_counts,
               cng2cngFullMat = ProbData.cng2cngFullMat, cng2index_dict = ProbData.cng2index_dict,
               w2w_samecng_fullmat=ProbData.w2w_samecng_fullmat, samecng_unigram_counts=ProbData.samecng_unigram_counts,
               v2c_fullMat = ProbData.v2c_fullMat)

goodFileDict = pickle.load(open('mergedGood_v3.p', 'rb'))
algo = SktWsegRWR(
            w2w_modelFunc = pb.get_w2w_mat, 
            t2t_modelFunc = pb.get_cng2cng_mat,
            v2c_modelFunc = pb.get_v2c_ranking,
            sameCng_modelFunc = pb.get_w2w_samecng_mat,
            partition=[0.25, 0.25, 0.25, 0.1]
        )

In [11]:
def loadSentence(fName, folderTag):
        # print('File: ', fName)
        try:
            dcsObj = pickleFixLoad('../Text Segmentation/DCS_pick/' + fName)           
            if folderTag == "C1020" :
                sentenceObj = pickleFixLoad('../TextSegmentation/corrected_10to20/' + fName)
            else:
                sentenceObj = pickleFixLoad('../TextSegmentation/Pickle_Files/' + fName)

        except (KeyError, EOFError, pickle.UnpicklingError) as e:
            return None, None
        return(sentenceObj, dcsObj)
    
def Accuracy(prediction, dcsObj):
#     solution = [rom_slp(c) for c in dcsObj.dcs_chunks]
    solution = [rom_slp(c) for arr in dcsObj.lemmas for c in arr]
#     ac = 100*sum(list(map(lambda x: x in solution, prediction)))/len(prediction)
    ac = 100*sum(list(map(lambda x: x in prediction, solution)))/len(solution)
    return ac

In [14]:
fullfs = ['60449.p','164056.p','379245.p']

In [65]:
np.set_printoptions(precision=6)
# for f in list(goodFileDict.keys())[90:91]:
# for f in list(goodFileDict.keys())[1:100]:
for f in fullfs:
    sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
    if(sentenceObj != None):
        result = algo.predict(sentenceObj, dcsObj)
        if(result != None):
            ac = Accuracy(result, dcsObj)
            solution = [rom_slp(c) for arr in dcsObj.lemmas for c in arr]
            print(result)
            print('vs')
            print(solution)
            print(f, "\t", ac)

['kftsna', 'tad', 'tad', 'BU', 'kf', 'KyA', 'asmad']
vs
['praBAva', 'saras', 'kftsna', 'AKyA', 'upakram']
60449.p 	 20.0
['tad', 'arDa', 'tad', 'eva', 'maDu', 'dA', 'as', 'rAj', 'raR']
vs
['maDura', 'eva', 'rAjya', 'tad', 'arDa', 'pradA']
164056.p 	 50.0
['as', 'putra', 'yuD', 'jaya', 'Ap', 'pARqava', 'prI']
vs
['prI', 'as', 'putra', 'yuD', 'jaya', 'Ap', 'pARqava']
379245.p 	 100.0


In [66]:
np.set_printoptions(precision=6)
# for f in list(goodFileDict.keys())[90:91]:
for f in list(goodFileDict.keys())[1:100]:
# for f in fullfs:
    sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
    if(sentenceObj != None):
        result = algo.predict(sentenceObj, dcsObj)
        if(result != None):
            ac = Accuracy(result, dcsObj)
            solution = [rom_slp(c) for arr in dcsObj.lemmas for c in arr]
#             print(result)
#             print('vs')
#             print(solution)
#             print(f, "\t", ac)

In [49]:
SeeSentence(sentenceObj)

prIto 'smi putra yuDyasva jayam Apnuhi pARqava   
Analyzing  prItaḥ
0 :  prItas ['prIta', 'prI'] [{'noun': ['nom. sg. m.']}, {'verb': ['pp.']}]
Analyzing  asmi
0 :  asmi ['as_1'] [{'verb': ['pr. [2] ac. sg. 1']}]
Analyzing  putra
0 :  putra ['putra'] [{'noun': ['voc. sg. m.']}]
Analyzing  yuDyasva
0 :  yuDyasva ['yuD_1'] [{'verb': ['imp. ps. sg. 2', 'imp. [4] md. sg. 2']}]
Analyzing  jayam
0 :  jayam ['jaya'] [{'noun': ['acc. sg. m.', 'acc. sg. n.', 'nom. sg. n.']}]
Analyzing  Apnuhi
0 :  Apnuhi ['Ap'] [{'verb': ['imp. [5] ac. sg. 2']}]
Analyzing  pARqava
0 :  pARqava ['pARqava'] [{'noun': ['voc. sg. m.']}]


In [50]:
SeeDCS(dcsObj)

prIto 'smi putra yuDyasva jayam Apnuhi pARqava   
[['prī'], ['as'], ['putra'], ['yudh'], ['jaya'], ['āp'], ['pāṇḍava']]
Lemmas: ['prI', 'as', 'putra', 'yuD', 'jaya', 'Ap', 'pARqava']
[['-190'], ['-11'], ['49'], ['-32'], ['69'], ['-32'], ['49']]


In [203]:
list(goodFileDict.keys())[13]

'327359.p'

In [209]:
lowfs

['179084.p',
 '212470.p',
 '141314.p',
 '312382.p',
 '371565.p',
 '20967.p',
 '178347.p',
 '289080.p',
 '334071.p',
 '407484.p',
 '209444.p']

In [ ]:
# 7941
# Results: 
# Mean:  65.7963242743
# Percentiles:  [   5.88   50.     66.67   85.71  100.  ]
